<a href="https://colab.research.google.com/github/dakilaledesma/Atomized_NN_Experiments/blob/main/augmentation_differences/TResNet_600_AugMix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
! unzip -q drive/MyDrive/UNC/H2022/orchidaceae_train.zip -d /content/

CPU times: user 127 ms, sys: 21.6 ms, total: 149 ms
Wall time: 20.2 s


In [ ]:
! git clone https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 10691, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 10691 (delta 109), reused 163 (delta 87), pack-reused 10469
Receiving objects: 100% (10691/10691), 20.38 MiB | 9.76 MiB/s, done.
Resolving deltas: 100% (7819/7819), done.


In [ ]:
! pip install git+https://github.com/mapillary/inplace_abn.git@v1.0.12

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/mapillary/inplace_abn.git (to revision v1.0.12) to /tmp/pip-req-build-sa2955d2
  Running command git clone -q https://github.com/mapillary/inplace_abn.git /tmp/pip-req-build-sa2955d2
  Running command git checkout -q 24fc791e6d4796a1639e7a5dce6fa67377e51a3e
  Created wheel for inplace-abn: filename=inplace_abn-1.0.12-cp37-cp37m-linux_x86_64.whl size=2324564 sha256=ea760b03f1cd93684f8df26d4f457385f61deb23a8927e413a647393e9c3bec8
  Stored in directory: /tmp/pip-ephem-wheel-cache-3cu0yuuz/wheels/df/ad/aa/7ec1eff8c9b56fb3fab10471e91c80bdf3b52036bf47727bae
Successfully built inplace-abn


In [ ]:
import fileinput
import sys

def replacement(file, previousw, nextw):
   for line in fileinput.input(file, inplace=1):
       line = line.replace(previousw, nextw)
       sys.stdout.write(line)

file = "/content/pytorch-image-models/timm/utils/checkpoint_saver.py"
replacement(file, "if os.path.exists(last_save_path):", "# if os.path.exists(last_save_path):")
replacement(file, "os.unlink(last_save_path)  # required for Windows support.", "# os.unlink(last_save_path)  # required for Windows support.")
replacement(file, "os.link(last_save_path, save_path)", "# os.link(last_save_path, save_path)")
replacement(file, "os.unlink(best_save_path)", "# os.unlink(best_save_path)")
replacement(file, "os.link(last_save_path, best_save_path)", "# os.link(last_save_path, best_save_path)")
replacement(file, "if os.path.exists(best_save_path):", "# if os.path.exists(best_save_path):")


In [ ]:
! python -u -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 --node_rank=0 ./pytorch-image-models/train.py orchidaceae_train --model tresnet_m_448 --opt adabelief --lr 0.001 --epochs 80 --aa augmix-m5-w4-mstd0.5 --aug-splits 4 --jsd-loss --decay-epochs 3 --cooldown-epochs 0 --weight-decay 1e-4 --sched cosine -b 16 --input-size 3 600 600 --num-classes=300 --vflip 0.5 --hflip 0.5 --amp --pretrained --output drive/MyDrive/UNC/H2022/output/

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Training with a single process on 1 GPUs.
Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-tresnet/tresnet_m_448-bc359d10.pth)
Model tresnet_m_448 created, param count:29954732
Data processing configuration for current model + dataset:
	input_size: (3, 600, 600)
	interpolation: bilinear
	mean: (0, 0, 0)
	std: (1, 1, 1)
	crop_pct: 0.875
Using native Torch AMP. Training in mixed precision.
Scheduled epochs: 80
/usr/local/lib/python3.7/dist-packages/torch/utils/data/d

In [ ]:
! python -u -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 --node_rank=0 ./pytorch-image-models/train.py orchidaceae_train --model tresnet_m_448 --opt adabelief --lr 0.001 --epochs 80 --aa augmix-m5-w4-mstd0.5 --aug-splits 4 --jsd-loss --decay-epochs 3 --cooldown-epochs 0 --weight-decay 1e-4 --sched cosine -b 16 --input-size 3 600 600 --num-classes=300 --vflip 0.5 --hflip 0.5 --amp --pretrained --output drive/MyDrive/UNC/H2022/output/ --resume drive/MyDrive/UNC/H2022/output/20220721-164211-tresnet_m_448-600/last.pth.tar

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Training with a single process on 1 GPUs.
Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-tresnet/tresnet_m_448-bc359d10.pth)
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-tresnet/tresnet_m_448-bc359d10.pth" to /root/.cache/torch/hub/checkpoints/tresnet_m_448-bc359d10.pth
Model tresnet_m_448 created, param count:29954732
Data processing configuration for current model + dataset:
	input_size: (3, 600, 600)
	interpolation: bi

In [ ]:
! python -u -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 --node_rank=0 ./pytorch-image-models/train.py orchidaceae_train --model tresnet_m_448 --opt adabelief --lr 0.001 --epochs 80 --aa augmix-m5-w4-mstd0.5 --aug-splits 4 --jsd-loss --decay-epochs 3 --cooldown-epochs 0 --weight-decay 1e-4 --sched cosine -b 16 --input-size 3 600 600 --num-classes=300 --vflip 0.5 --hflip 0.5 --amp --pretrained --output drive/MyDrive/UNC/H2022/output/ --resume drive/MyDrive/UNC/H2022/output/20220722-171508-tresnet_m_448-600/last.pth.tar

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:186: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  FutureWarning,
Training with a single process on 1 GPUs.
Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-tresnet/tresnet_m_448-bc359d10.pth)
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-tresnet/tresnet_m_448-bc359d10.pth" to /root/.cache/torch/hub/checkpoints/tresnet_m_448-bc359d10.pth
Model tresnet_m_448 created, param count:29954732
Data processing configuration for current model + dataset:
	input_size: (3, 600, 600)
	interpolation: bi